In [1]:
%pip install datasets
%pip install nltk
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



# Part 0. Data Prepraration

In [2]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset ['train']
validation_dataset = dataset ['validation']
test_dataset = dataset ['test']

c:\Users\JJWX\anaconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
len(train_dataset['text'])

8530

# Part 1. Preparing Word Embeddings

#### (a) What is the size of the vocabulary formed from your training data?

In [4]:
import nltk
import numpy as np
nltk.download('punkt')
vocab = set()
for text in train_dataset['text']:
    ls = nltk.word_tokenize(text)
    for word in ls:
        if word.isalpha(): vocab.add(word)
#Size of the vocabulary: 15812

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JJWX\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
print("Size of the vocabulary:", len(vocab))

Size of the vocabulary: 15812


#### (b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

In [6]:
import gensim.downloader as api
for key in api.info()['models'].keys():
    print(key)

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [7]:
model = api.load("glove-wiki-gigaword-100")

In [8]:
oov_words = set()
for word in vocab:
    if word not in model:
        oov_words.add(word)

print("Number of OOV words:", len(oov_words))

Number of OOV words: 550


#### (c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.

In [9]:
# Group any words that are not in the model into a single token
def wordtovec(word):
    if word in model:
        return model[word]
    else:
        return np.zeros(model.vector_size)

# Part 2. Model Training & Evaluation - RNN

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import random

In [14]:
vocab_size = len(model.index_to_key) + 1
embedding_dim = model.vector_size
word_index = {word: index+1 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [15]:
for word, idx in word_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]

In [16]:
def tokenize(text, word_index):
    ls = nltk.word_tokenize(text)
    return [word_index[word] for word in ls if word in word_index]

X_train = [tokenize(text, word_index) for text in train_dataset['text']]
X_val = [tokenize(text, word_index) for text in validation_dataset['text']]
X_test = [tokenize(text, word_index) for text in test_dataset['text']]
max_length = max(len(seq) for seq in X_train)

In [17]:
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

In [18]:
y_train = np.array(train_dataset['label'])
y_val = np.array(validation_dataset['label'])
y_test = np.array(test_dataset['label'])

Model Training - Grid Search

In [171]:
from tensorflow.keras.callbacks import Callback
best_accuracy = {}
class CustomCallback(Callback):
    accuracy = 0
    cur_key = ""
    epochs = 0
    optimizer = ""
    batch_size = 0
    best_model = None
    lr = 0
    def on_train_begin(self, logs=None):
        self.accuracy = 0

    def on_train_end(self, logs=None):
        global best_accuracy
        if self.accuracy > best_accuracy.get("accuracy", 0):
            best_accuracy = {
                "accuracy": self.accuracy,
                "epoch": self.epochs,
                "optimizer": self.optimizer,
                "batch_size": self.batch_size,
                "lr": self.lr
            }
            print("Saved best accuracy for current run:", self.accuracy, "at epoch", self.epochs)
            self.best_model.save(filepath="best_model.keras")
        print("Run completed on:")
        print(self.cur_key)
        print("Best accuracy for current run:", self.accuracy, "at epoch", self.epochs)
        print("Training ended")
    
    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs['val_accuracy']
        if val_accuracy > self.accuracy:
            self.accuracy = val_accuracy
            self.epochs = epoch
            self.best_model = self.model

    def set_key(self, optimizer, batch_size, lr):
        self.optimizer = optimizer
        self.batch_size = batch_size
        self.lr = lr
        self.cur_key = f"optimizer: {optimizer}, batch_size: {batch_size}, lr: {lr}"

In [170]:
from tensorflow.keras.layers import Dropout
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    custom_callback = CustomCallback()
    custom_callback.set_key(optimizer, batch_size, lr)
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(32, return_sequences=False),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[custom_callback, early_stopping]
    )
    return model, history

In [34]:
for batch_size in [16, 32, 64, 128]:
    for lr in [0.005, 0.01, 0.05, 0.1]:
        for optimizer in ['adam', 'sgd', 'rmsprop', 'adagrad']:
            train_model(optimizer, 100, batch_size, lr)

Epoch 1/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5024 - loss: 0.6963 - val_accuracy: 0.5000 - val_loss: 0.6938
Epoch 2/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5022 - loss: 0.6941 - val_accuracy: 0.4991 - val_loss: 0.6935
Epoch 3/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5014 - loss: 0.6936 - val_accuracy: 0.5169 - val_loss: 0.6924
Epoch 4/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4911 - loss: 0.6968 - val_accuracy: 0.5000 - val_loss: 0.6936
Epoch 5/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4946 - loss: 0.6936 - val_accuracy: 0.5009 - val_loss: 0.6934
Epoch 6/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.4982 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6934
Saved best accuracy for current run: 0.5168855786323547 at epoch 2
Run completed on:
optimizer: adam, batch_size: 16, lr: 0.005
Best accuracy for current run: 0.5168855786323547 at epoch 2
Training ended
Epoch 1/100
534/534 ━━

In [35]:
best_accuracy

{'accuracy': 0.7016885280609131,
 'epoch': 35,
 'optimizer': 'adagrad',
 'batch_size': 64,
 'lr': 0.01}

Best model is trained with Optimizer: adagrad, Epoch: 35, Batch_size: 64, Learning_rate: 0.01

In [172]:
model, history = train_model("adagrad", 40, 64, 0.01)

Epoch 1/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4966 - loss: 0.7008 - val_accuracy: 0.5009 - val_loss: 0.6942
Epoch 2/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5049 - loss: 0.6948 - val_accuracy: 0.5009 - val_loss: 0.6938
Epoch 3/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4948 - loss: 0.6958 - val_accuracy: 0.5113 - val_loss: 0.6934
Epoch 4/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5110 - loss: 0.6936 - val_accuracy: 0.5000 - val_loss: 0.6942
Epoch 5/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5169 - loss: 0.6921 - val_accuracy: 0.5047 - val_loss: 0.6940
Epoch 6/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5201 - loss: 0.6922 - val_accuracy: 0.4953 - val_loss: 0.6941
Saved best accuracy for current run: 0.5112570524215698 at epoch 2
Run completed on:
optimizer: adagrad, batch_size: 64, lr: 0.01
Best accuracy for current run: 0.5112570524215698 at epoch 2
Training ended


In [173]:
best_model = tf.keras.models.load_model("best_model.keras")
accuracy = best_model.evaluate(X_val, y_val)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4740 - loss: 0.6946
Test accuracy: 0.4953095614910126


In [38]:
best_model.get_compile_config()

{'optimizer': {'module': 'keras.optimizers',
  'class_name': 'Adagrad',
  'config': {'name': 'adagrad',
   'learning_rate': 0.009999999776482582,
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'loss_scale_factor': None,
   'gradient_accumulation_steps': None,
   'initial_accumulator_value': 0.1,
   'epsilon': 1e-07},
  'registered_name': None},
 'loss': 'binary_crossentropy',
 'loss_weights': None,
 'metrics': ['accuracy'],
 'weighted_metrics': None,
 'run_eagerly': False,
 'steps_per_execution': 1,
 'jit_compile': False}

#### Mean pooling

In [40]:
from tensorflow.keras.layers import AveragePooling1D, GlobalAveragePooling1D
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_mean.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        GlobalAveragePooling1D(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [41]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
131/134 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4601 - loss: 0.6998
Epoch 1: val_accuracy improved from -inf to 0.48311, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4605 - loss: 0.6997 - val_accuracy: 0.4831 - val_loss: 0.6947
Epoch 2/100
128/134 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4787 - loss: 0.6953
Epoch 2: val_accuracy improved from 0.48311 to 0.49062, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4793 - loss: 0.6953 - val_accuracy: 0.4906 - val_loss: 0.6933
Epoch 3/100
128/134 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4952 - loss: 0.6938
Epoch 3: val_accuracy improved from 0.49062 to 0.50750, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4956 - loss: 0.6938 - val_accuracy: 0.5075 - val_loss: 0.6922
Epoch 4/100
130/134 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5123 - loss: 0.6927
Epoch 4: val_accuracy improved fro

In [43]:
best_model = tf.keras.models.load_model("model_mean.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6029 - loss: 0.6595
Test accuracy: 0.6594746708869934


#### Max pooling

In [166]:
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPooling1D
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_max.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        GlobalMaxPooling1D(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [167]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
130/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5096 - loss: 0.7060
Epoch 1: val_accuracy improved from -inf to 0.52439, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 47ms/step - accuracy: 0.5096 - loss: 0.7057 - val_accuracy: 0.5244 - val_loss: 0.6920
Epoch 2/100
128/134 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5311 - loss: 0.6902
Epoch 2: val_accuracy improved from 0.52439 to 0.54034, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5311 - loss: 0.6902 - val_accuracy: 0.5403 - val_loss: 0.6882
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5482 - loss: 0.6856
Epoch 3: val_accuracy improved from 0.54034 to 0.54597, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5482 - loss: 0.6856 - val_accuracy: 0.5460 - val_loss: 0.6838
Epoch 4/100
129/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5686 - loss: 0.6803
Epoch 4: val_accuracy improved from 0

In [168]:
best_model = tf.keras.models.load_model("model_max.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6999 - loss: 0.5836
Test accuracy: 0.7298311591148376


#### Dense layer

In [55]:
from tensorflow.keras.layers import Flatten
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_dense.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        Flatten(),
        Dense(62, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [56]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.5301 - loss: 0.7015
Epoch 1: val_accuracy improved from -inf to 0.54128, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 26s 185ms/step - accuracy: 0.5303 - loss: 0.7014 - val_accuracy: 0.5413 - val_loss: 0.6979
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.6130 - loss: 0.6606
Epoch 2: val_accuracy improved from 0.54128 to 0.56848, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 145ms/step - accuracy: 0.6131 - loss: 0.6605 - val_accuracy: 0.5685 - val_loss: 0.7107
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.6440 - loss: 0.6323
Epoch 3: val_accuracy improved from 0.56848 to 0.60413, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 147ms/step - accuracy: 0.6440 - loss: 0.6322 - val_accuracy: 0.6041 - val_loss: 0.6806
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.6822 - loss: 0.6002
Epoch 4: val_accu

In [57]:
best_model = tf.keras.models.load_model("model_dense.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6231 - loss: 0.7368
Test accuracy: 0.7232645153999329


: 

# Part 3. Enhancement


#### 1. Instead of keeping the word embeddings fixed, now update the word embeddings (the same way as model parameters) during the training process.


In [ ]:
tf.random.set_seed(0)
np.random.seed(0)
random.seed(0)
model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              trainable=True),
    SimpleRNN(16, return_sequences=False),
    Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01) #Static learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping]
)

#### 2. As discussed in Question 1(c), apply your solution in mitigating the influence of OOV words and train your model again.

In [19]:
vocab_size = len(model.index_to_key) + 2 # 0 is reserved for padding, 1 is reserved for OOV
embedding_dim = model.vector_size
word_index = {word: index+2 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [20]:
for word, idx in word_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]

In [21]:
def tokenize(text, word_index):
    ls = nltk.word_tokenize(text)
    return [word_index[word] if word in word_index else 1 for word in ls]

X_train = [tokenize(text, word_index) for text in train_dataset['text']]
X_val = [tokenize(text, word_index) for text in validation_dataset['text']]
X_test = [tokenize(text, word_index) for text in test_dataset['text']]
max_length = max(len(seq) for seq in X_train)

In [22]:
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

In [23]:
y_train = np.array(train_dataset['label'])
y_val = np.array(validation_dataset['label'])
y_test = np.array(test_dataset['label'])

In [29]:
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_oov.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [30]:
train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5133 - loss: 0.6945
Epoch 1: val_accuracy improved from -inf to 0.52439, saving model to model_oov.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5132 - loss: 0.6945 - val_accuracy: 0.5244 - val_loss: 0.6933
Epoch 2/100
126/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5323 - loss: 0.6905
Epoch 2: val_accuracy did not improve from 0.52439
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5322 - loss: 0.6906 - val_accuracy: 0.5206 - val_loss: 0.6940
Epoch 3/100
132/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5591 - loss: 0.6857
Epoch 3: val_accuracy improved from 0.52439 to 0.53377, saving model to model_oov.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5589 - loss: 0.6858 - val_accuracy: 0.5338 - val_loss: 0.6915
Epoch 4/100
131/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5766 - loss: 0.6794
Epoch 4: val_accuracy improved from 0.53377 to 0.55253, saving model to mode

(<Sequential name=sequential_1, built=True>,
 <keras.src.callbacks.history.History at 0x1d5b9a1ec90>)

In [31]:
best_model = tf.keras.models.load_model("model_oov.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6786 - loss: 0.5732
Test accuracy: 0.6763602495193481


#### 3. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a biLSTM model and a biGRU model, incorporating recurrent computations in both directions and stacking multiple layers if possible

BiLSTM

In [67]:
from tensorflow.keras.layers import Bidirectional, LSTM
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is frozen
        Bidirectional(LSTM(16, return_sequences=True)),
        Bidirectional(LSTM(16, return_sequences=True)),
        Bidirectional(LSTM(16, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback]
    )
    return model, history

In [68]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.5006 - loss: 0.6935
Epoch 1: val_accuracy improved from -inf to 0.58443, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 33s 202ms/step - accuracy: 0.5006 - loss: 0.6935 - val_accuracy: 0.5844 - val_loss: 0.6909
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.5479 - loss: 0.6905
Epoch 2: val_accuracy improved from 0.58443 to 0.60038, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 24s 180ms/step - accuracy: 0.5479 - loss: 0.6905 - val_accuracy: 0.6004 - val_loss: 0.6876
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.5880 - loss: 0.6862
Epoch 3: val_accuracy improved from 0.60038 to 0.61632, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 25s 188ms/step - accuracy: 0.5880 - loss: 0.6862 - val_accuracy: 0.6163 - val_loss: 0.6805
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.6149 - loss: 0.6756
Epoch 4:

In [65]:
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)        │ (None, 62, 100)        │    40,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_11                │ (None, 62, 32)         │        14,976 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_12                │ (None, 62, 32)         │         6,272 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_13                │ (None, 32)             │         6,272 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,055,508 (305.39 MB)

 Trainable params: 40,027,753 (152.69 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 40,027,755 (152.69 MB)

In [66]:
best_model = tf.keras.models.load_model("model_combined.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6798 - loss: 0.5810
Test accuracy: 0.7429643273353577


BiGRU

In [37]:
from tensorflow.keras.layers import Bidirectional, GRU
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is frozen
        Bidirectional(GRU(16, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [38]:
train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.5007 - loss: 0.7018
Epoch 1: val_accuracy improved from -inf to 0.50750, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 28s 176ms/step - accuracy: 0.5007 - loss: 0.7017 - val_accuracy: 0.5075 - val_loss: 0.6927
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.5439 - loss: 0.6876
Epoch 2: val_accuracy improved from 0.50750 to 0.55159, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 24s 182ms/step - accuracy: 0.5440 - loss: 0.6876 - val_accuracy: 0.5516 - val_loss: 0.6857
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.5801 - loss: 0.6800
Epoch 3: val_accuracy improved from 0.55159 to 0.55910, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 24s 179ms/step - accuracy: 0.5801 - loss: 0.6800 - val_accuracy: 0.5591 - val_loss: 0.6803
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.5972 - loss: 0.6738
Epoch 4:

(<Sequential name=sequential_4, built=True>,
 <keras.src.callbacks.history.History at 0x1d5d1d63690>)

In [39]:
best_model = tf.keras.models.load_model("model_combined.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7120 - loss: 0.5646
Test accuracy: 0.7532833218574524


#### 4. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a Convolutional Neural Network (CNN) to produce sentence representations and perform sentiment classification

In [50]:
from tensorflow.keras.layers import Convolution1D, Flatten

def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is trainable
        Convolution1D(16, kernel_size=3, activation='relu'),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [52]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.4938 - loss: 0.7037
Epoch 1: val_accuracy improved from -inf to 0.53846, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 19s 133ms/step - accuracy: 0.4939 - loss: 0.7036 - val_accuracy: 0.5385 - val_loss: 0.6903
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.5456 - loss: 0.6871
Epoch 2: val_accuracy improved from 0.53846 to 0.58161, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 18s 131ms/step - accuracy: 0.5457 - loss: 0.6870 - val_accuracy: 0.5816 - val_loss: 0.6789
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.5965 - loss: 0.6690
Epoch 3: val_accuracy improved from 0.58161 to 0.60694, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 18s 131ms/step - accuracy: 0.5966 - loss: 0.6690 - val_accuracy: 0.6069 - val_loss: 0.6650
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.6333 - loss: 0.6457
Epoch 4:

In [53]:
best_model = tf.keras.models.load_model("model_combined.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6092 - loss: 0.7034
Test accuracy: 0.7101313471794128


#### 5 

In [25]:
from tensorflow.keras.layers import MultiHeadAttention, Input, Bidirectional, LSTM, Dropout, Dense, Lambda, GlobalMaxPooling1D, Flatten, GlobalAveragePooling1D, GRU
from tensorflow.keras.models import Model

def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    )
    
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )

    input_layer = Input(shape=(max_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=True)(input_layer)
    
    gru_output = Bidirectional(GRU(64, return_sequences=True))(embedding_layer)
    gru_output = Dropout(0.5)(gru_output)
    gru_output = Bidirectional(GRU(32, return_sequences=True))(gru_output)
    gru_output = Dropout(0.5)(gru_output)
    gru_output = Bidirectional(GRU(16, return_sequences=True))(gru_output)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=32)(gru_output, gru_output)
    attention_output = Dropout(0.5)(attention_output)

    max_output = GlobalMaxPooling1D()(attention_output)
    mean_output = GlobalAveragePooling1D()(attention_output)
    gru_output = gru_output[:, -1, :]  # Get the last output of the GRU layer
    concat_output = tf.keras.layers.Concatenate(axis=1)([max_output, mean_output, gru_output])  # Concatenate the outputs of the two layers 
    output_layer = Dense(1, activation='sigmoid')(concat_output)  # Final output layer for binary classification

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',     # Monitor validation loss
        factor=0.5,             # Factor to reduce the learning rate
        patience=3,             # Number of epochs with no improvement to wait
        min_lr=1e-6             # Minimum learning rate limit
    )
    # Set optimizer
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=reduce_lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=reduce_lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=reduce_lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)

    # Compile the model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping, reduce_lr]
    )

    return model, history

In [26]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.5055 - loss: 0.6933
Epoch 1: val_accuracy improved from -inf to 0.50657, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 42s 264ms/step - accuracy: 0.5055 - loss: 0.6933 - val_accuracy: 0.5066 - val_loss: 0.6934 - learning_rate: 0.0100
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - accuracy: 0.5079 - loss: 0.6929
Epoch 2: val_accuracy improved from 0.50657 to 0.51032, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 27s 202ms/step - accuracy: 0.5079 - loss: 0.6929 - val_accuracy: 0.5103 - val_loss: 0.6931 - learning_rate: 0.0100
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.5169 - loss: 0.6921
Epoch 3: val_accuracy improved from 0.51032 to 0.52720, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 28s 206ms/step - accuracy: 0.5169 - loss: 0.6921 - val_accuracy: 0.5272 - val_loss: 0.6927 - learning_rate: 0.0100
Epoch 4/100
134/134 ━━━━━

In [27]:
model = tf.keras.models.load_model("model_combined.keras")
accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7040 - loss: 0.6004
Test accuracy: 0.7213883399963379


In [155]:
model.summary()

Model: "functional_53"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_45      │ (None, 62)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_45        │ (None, 62, 100)   │ 40,000,100 │ input_layer_45[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_39    │ (None, 62, 64)    │     25,728 │ embedding_45[0][… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 62, 64)    │          0 │ bidirectional_39… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_40    │ (None, 62, 64)    │     18,816 │ dropout_5[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 62, 64)    │          0 │ bidirectional_40… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_41    │ (None, 62, 64)    │     18,816 │ dropout_6[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 64)        │          0 │ bidirectional_41… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ bidirectional_41… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 64)        │          0 │ bidirectional_41… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_28      │ (None, 192)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_average_p… │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 1)         │        193 │ concatenate_28[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 80,127,308 (305.66 MB)

 Trainable params: 40,063,653 (152.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 40,063,655 (152.83 MB)